In [21]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('bmh')
os.chdir('/home/seidi/Repositories/mestrado_final/')

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)

import utils
# Reading data
mb = utils.read_mb()
mb = utils.read_int_mb()
mf = utils.read_mf()
# mb['Recall'] = mb['Recall'].round(decimals=2)

In [22]:
adbis_scores_path = 'data/results/info_sys_interpolated/adbis/scores.csv'
adbis_preds_path = 'data/results/info_sys_interpolated/adbis/predictions.csv'
# # adbis_scores = pd.read_csv(adbis_scores_path)
# # adbis_scores[(adbis_scores.approach == 'tmm_s') & (adbis_scores.target == 'Recall')]
# # adbis_scores.drop_duplicates(['base', 'target', 'approach'], inplace=True)
# # adbis_scores = adbis_scores[adbis_scores.approach == 'tmm_s']
    
adbis_preds = pd.read_csv(adbis_preds_path)
adbis_preds = adbis_preds[adbis_preds.target != 'IndexTime']
adbis_preds.drop_duplicates(['base', 'target', 'approach', 'NN', 'R', 'graph_type', 'nr_inst', 'k_searching'], inplace=True)

adbis_preds = adbis_preds.rename(columns={'approach': 'method', 'true': 'y_true', 'pred': 'y_pred', 'NN': 'IndexParams', 'R': 'QueryTimeParams'})
adbis_preds.method = adbis_preds.method.apply(lambda x: x.replace('_', ''))
adbis_preds.nr_inst = adbis_preds.nr_inst.apply(np.exp)
adbis_preds.base = adbis_preds.base.apply(lambda x: x.split('_')[0])
adbis_preds.base = adbis_preds.base + '_' + adbis_preds.nr_inst.round().astype(str).apply(lambda x: x.split('.')[0])
# adbis_preds.base = adbis_preds.base.astype(str) + '_' + adbis_preds.nr_inst.round().astype(int).astype(str)

In [23]:
scores_path = 'data/results/info_sys_interpolated/clustering_tuned/scores.csv'
preds_path = 'data/results/info_sys_interpolated/clustering_tuned/predictions.csv'
scores = pd.read_csv(scores_path)
scores = scores[scores.feature_selection_method == 'rf']
preds = pd.read_csv(preds_path)
preds = preds[preds.target != 'IndexTime']
preds = preds[preds.feature_selection_method == 'rf']

scores.method = scores.method.apply(lambda x: x + '+')
preds.method = preds.method.apply(lambda x: x + '+')
preds = pd.concat([preds, adbis_preds])
preds.drop(['feature_selection_method', 'nr_inst'], axis=1, inplace=True)

In [24]:
# Melt: columns to rows; Pivot: rows to columns
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error
k_searching = 1
recommends = preds[preds.k_searching == k_searching].pivot(index=['base', 'IndexParams', 'QueryTimeParams', 'graph_type', 'method', 'k_searching'], columns='target', values=['y_true', 'y_pred']).copy()
recommends.columns = [ '_'.join(x) for x in recommends.columns ]
recommends.reset_index(inplace=True)
err = recommends.groupby(['method']).apply(lambda x: mean_absolute_error(x.y_true_Recall, 
x.y_pred_Recall))
print(err)
# print(recommends.head()[['base', 'y_pred_Recall', 'method']])
err = err.to_dict()
f = lambda x: x.y_pred_Recall - err[x.method]
recommends['y_pred_Recall'] = recommends[['method', 'y_pred_Recall']].apply(f, axis=1)
# print(recommends.head()[['base', 'y_pred_Recall', 'method']])

recommends[['y_true_Recall', 'y_pred_Recall']] = recommends[['y_true_Recall', 'y_pred_Recall']].round(decimals=2)
recommends = recommends.rename(columns={'method': 'approach'})

method
gmm       0.039788
gmm+      0.028483
tmmgs     0.029490
tmmgs+    0.019581
tmms      0.024069
tmms+     0.016634
dtype: float64


In [25]:
'''
    Pegar valor bom frequente, independente do dataset, afinal nós não estamos considerando nenhuma informação do dataset de entrada (um default da biblioteca):
    - filtrar os que atendem o requisito de recall minimo, considerando todos os datasets, todos os grafos e todos os parametros
    - identificar os top 10% por métrica de otimização
    - identificar o grafo mais frequente
    - parâmetros:
        - a) são escolhidos através da média do NN e a média do R para esse NN (recomendação "justa", espera-se que não atinja o requisito em 50% dos casos). Por que NN primeiro? Porque o R é ajustável tendo o grafo construído, o NN requer reconstrução do grafo
        - b) são escolhidos através do máximo NN e a média do R para esse NN (recomendação "frouxa" com mais chances de atingir o requisito)
'''
# Recall .90 otimizando QT 
atende_requisito = mb[(mb['Recall'] >= 0.99) & (mb['QueryTimeParams'] <= 120)].copy() # NN não está multiplicado pro NSW
top10_qt = atende_requisito.sort_values(by='IndexParams').iloc[:int(len(atende_requisito)/10), :]
# top10_nn = atende_requisito.sort_values(by='IndexParams').iloc[:int(len(atende_requisito)/10), :]
moda_qt = top10_qt.graph_type.mode().values[0]
top10_qt_moda_grafo = top10_qt[top10_qt.graph_type == moda_qt]

# # NN Ajustado
# top10_nn_qt = top10_qt_moda_grafo.IndexParams.mean()
# nn_unique = top10_qt_moda_grafo.IndexParams.unique()
# diff_nn = [abs(top10_nn_qt - nn) for nn in nn_unique]
# top10_nn_qt = pd.DataFrame({
#     'diff': diff_nn,
#     'nn_unique': nn_unique
# }).sort_values(by='diff').iloc[0, 1]
# Frouxa
top10_nn_qt = top10_qt_moda_grafo.IndexParams.max()

# RR Ajustado
# top10_rr_qt = top10_qt_moda_grafo[top10_qt_moda_grafo.IndexParams == top10_nn_qt].QueryTimeParams.mean()
# rr_unique = top10_qt_moda_grafo.QueryTimeParams.unique()
# diff_rr = [abs(top10_rr_qt - rr) for rr in rr_unique]
# top10_rr_qt = pd.DataFrame({
#     'diff': diff_rr,
#     'nn_unique': rr_unique
# }).sort_values(by='diff').iloc[0, 1]
# # RR frouxo
top10_rr_qt = top10_qt_moda_grafo[top10_qt_moda_grafo.IndexParams == top10_nn_qt].QueryTimeParams.max()

print(top10_nn_qt)
print(top10_rr_qt)
print(moda_qt)

35.0
120.0
0.0


In [26]:
REAL_BASES = [
    'texture_67940', 'sift_999900', 'moments_67940',
    'mnist121d_69900', 'fashion_69900',
    'colorHisto_67940', 'mnist_69900', 
    'cophir282_999900', 'cophir64_999900',
    'base71_999900',
    
    'mnist121d_16000', 'mnist121d_32000', 
    'mnist_16000', 'mnist_32000', 
    'cophir282_500000', 'cophir282_100000',
    'cophir64_500000', 'cophir64_100000', 
    'base71_100000', 'base71_500000',
    'texture_16000', 'texture_32000',
    'sift_500000', 'sift_100000',
    'moments_16000', 'moments_32000',
    'fashion_16000', 'fashion_32000',
    'colorHisto_16000', 'colorHisto_32000',
]
average_params = {
    'IndexParams': {
        'Heavy': {
            0.90: {'nn': 20, 'rr': 60, 'gt': 0},
            0.95: {'nn': 20, 'rr': 60, 'gt': 0},
            0.99: {'nn': 25, 'rr': 60, 'gt': 0} 
        },
        'Loose': {
            0.90: {'nn': 30, 'rr': 120, 'gt': 0},
            0.95: {'nn': 30, 'rr': 120, 'gt': 0},
            0.99: {'nn': 35, 'rr': 120, 'gt': 0}
        }
    },
    'QueryTime': {
        'Heavy': {
            0.90: {'nn': 70, 'rr': 6, 'gt': 1},
            0.95: {'nn': 60, 'rr': 5, 'gt': 0},
            0.99: {'nn': 70, 'rr': 7, 'gt': 0} 
        },
        'Loose': {
            0.90: {'nn': 100, 'rr': 10, 'gt': 1},
            0.95: {'nn': 100, 'rr': 8, 'gt': 0},
            0.99: {'nn': 100, 'rr': 10, 'gt': 0}
        }
    },
    'DistComp': {
        'Heavy': {
            0.90: {'nn': 65, 'rr': 5, 'gt': 2},
            0.95: {'nn': 60, 'rr': 4, 'gt': 0},
            0.99: {'nn': 70, 'rr': 7, 'gt': 0} 
        },
        'Loose': {
            0.90: {'nn': 100, 'rr': 8, 'gt': 2},
            0.95: {'nn': 100, 'rr': 5, 'gt': 0},
            0.99: {'nn': 100, 'rr': 10, 'gt': 0}
        }
    },
}
def get_baseline(mb, optmizing_name, optmizing, approach, required_recall):
    tmp = mb[
        (mb.IndexParams == average_params[optmizing][approach][required_recall]['nn']) &
        (mb.QueryTimeParams == average_params[optmizing][approach][required_recall]['rr']) & 
        (mb.graph_type == average_params[optmizing][approach][required_recall]['gt']) & 
        (mb.k_searching == k_searching) &
        (mb.index.isin(REAL_BASES))
        # (recommends.approach == 'gmm+')
    ]
    tmp.reset_index(inplace=True)
    return pd.DataFrame(
        data={
            'base': tmp.base, 
            'IndexParams': tmp.IndexParams, 
            'QueryTimeParams': tmp.QueryTimeParams,
            'graph_type': tmp.graph_type,
            'approach': approach,
            'k_searching': tmp.k_searching,
            'y_true_DistComp': tmp.DistComp,
            'y_true_QueryTime': tmp.QueryTime,
            'y_true_Recall': tmp.Recall,
            'y_pred_DistComp': tmp.DistComp, 
            'y_pred_QueryTime': tmp.QueryTime,
            'y_pred_Recall': tmp.Recall,
            'optmizing': optmizing_name[optmizing],
            'required_recall': required_recall
        },
    )

In [27]:
NN = [5., 25., 70., 100.]
R = [1., 10., 40., 120.]
pred_recall = 'y_pred_Recall'
true_recall = 'y_true_Recall'

def get_best(df, recall='y_true_Recall', optmizing='y_true_QueryTime', method_name='gs', optmizing_name='qt', required_recall=0.9):
    ix = df[df[recall] >= required_recall].groupby(['base'])[optmizing].idxmin().values
    tmp = df.loc[ix, :].copy()
    tmp['approach'] = method_name
    tmp['optmizing'] = optmizing_name
    return tmp

def get_rec(recommends, required_recall, optmizing='y_pred_QueryTime', optmizing_name='qt'):
    cols = ['base', 'IndexParams', 'QueryTimeParams', 'graph_type', 'approach', 'y_pred_Recall', 'y_true_Recall']
    ix = recommends[recommends['y_pred_Recall'] >= required_recall].groupby(['base', 'approach'])[optmizing].nsmallest(1).index.get_level_values(2).values
    tmp = recommends.loc[ix, cols].groupby(['base', 'approach'])[['graph_type', 'QueryTimeParams', 'IndexParams']].median().reset_index()
    index = ['base','approach','graph_type','QueryTimeParams','IndexParams']
    tmp = recommends.join(tmp.set_index(index), on=index, how='inner', lsuffix='l_')
    tmp['optmizing'] = optmizing_name
    return tmp

cols = np.append(recommends.columns, np.array(['optmizing', 'required_recall']))
final_recommendations = pd.DataFrame(columns=cols)
optmizing = 'IndexParams'
optmizing_name = {
    'QueryTime': 'qt',
    'IndexParams': 'nn',
    'DistComp': 'dt'
}

flag = False
for optmizing in ['QueryTime', 'IndexParams', 'DistComp']:
    for required_recall in [0.90, 0.95, 0.99]:
        # GS
        gs = recommends.copy()
        gs = gs[(gs.IndexParams.isin(NN)) & (gs.QueryTimeParams.isin(R))]
        tmp = get_best(gs, recall=true_recall, optmizing='y_true_' + optmizing, method_name='GridSearch', optmizing_name=optmizing_name[optmizing], required_recall=required_recall) if optmizing != 'IndexParams' else get_best(gs, recall=true_recall, optmizing=optmizing, method_name='GridSearch', optmizing_name=optmizing_name[optmizing], required_recall=required_recall)
        tmp['required_recall'] = required_recall
        tmp['approach'] = 'GridSearch'
        final_recommendations = pd.concat([final_recommendations.loc[:, cols], tmp.loc[:, cols]])
        
        # OPTIMAL 
        tmp = get_best(recommends.copy(), recall=true_recall, optmizing='y_true_' + optmizing, method_name='Optimal', optmizing_name=optmizing_name[optmizing], required_recall=required_recall) if optmizing != 'IndexParams' else get_best(recommends, recall=true_recall, optmizing=optmizing, method_name='Optimal', optmizing_name=optmizing_name[optmizing], required_recall=required_recall)
        tmp['required_recall'] = required_recall 
        tmp['approach'] = 'Optimal'
        final_recommendations = pd.concat([final_recommendations.loc[:, cols], tmp.loc[:, cols]])
        
        # AVERAGE PARAMS
        # tmp = mb[
        #     (mb.IndexParams == average_params[optmizing]['justo'][required_recall]['nn']) &
        #     (mb.QueryTimeParams == average_params[optmizing]['justo'][required_recall]['rr']) &
        #     (mb.graph_type == average_params[optmizing]['justo'][required_recall]['gt']) &
        #     (mb.k_searching == k_searching)
        # ].copy()

        # tmp['approach'] = 'Heavy'
        # tmp['optmizing'] = optmizing_name[optmizing]
        # tmp['required_recall'] = required_recall
        # final_recommendations = pd.concat([final_recommendations, tmp])
        tmp = get_baseline(mb, optmizing_name, optmizing, 'Heavy', required_recall)
        final_recommendations = pd.concat([final_recommendations, tmp])
                
        # AVERAGE PARAMS
        # tmp = recommends[
        #     (recommends.IndexParams == average_params[optmizing]['frouxo'][required_recall]['nn']) &
        #     (recommends.QueryTimeParams == average_params[optmizing]['frouxo'][required_recall]['rr']) &
        #     (recommends.graph_type == average_params[optmizing]['frouxo'][required_recall]['gt']) &
        #     (recommends.approach == 'gmm+')
        # ].copy()

        # tmp['approach'] = 'Light'
        # tmp['optmizing'] = optmizing_name[optmizing]
        # tmp['required_recall'] = required_recall
        tmp = get_baseline(mb, optmizing_name, optmizing, 'Loose', required_recall)
        tmp['approach'] = 'Light'
        final_recommendations = pd.concat([final_recommendations, tmp])
                
        # RECOMMENDED
        if required_recall == 0.99:
            # required_recall -= error
            f = lambda x: x.y_pred_Recall + err[x.approach]
            recommends['y_pred_Recall'] = recommends[['approach', 'y_pred_Recall']].apply(f, axis=1)
            flag = True
        tmp = get_rec(recommends.copy(), required_recall, optmizing='y_pred_' + optmizing, optmizing_name=optmizing_name[optmizing]) if optmizing != 'IndexParams' else get_rec(recommends, required_recall, optmizing=optmizing, optmizing_name=optmizing_name[optmizing])

        if flag:
            flag = False
            f = lambda x: x.y_pred_Recall - err[x.approach]
            recommends['y_pred_Recall'] = recommends[['approach', 'y_pred_Recall']].apply(f, axis=1)
        tmp['required_recall'] = required_recall
        final_recommendations = pd.concat([final_recommendations, tmp])

final_recommendations.loc[:, ['y_true_QueryTime', 'y_true_DistComp', 'y_pred_QueryTime']] = final_recommendations.loc[:, ['y_true_QueryTime', 'y_true_DistComp', 'y_pred_QueryTime']].apply(np.exp)
# final_recommendations.base = final_recommendations.base.apply(lambda x: x.split('_')[0])
final_recommendations = final_recommendations.rename(columns={
    'y_true_QueryTime': 'Query Time (ms)',
    'y_true_Recall': 'Recall',
    'IndexParams': 'NN',
    'QueryTimeParams': 'R',
    'y_true_DistComp': 'Distance Computations',
    'method': 'Method',
    'base': 'Dataset'
})
final_recommendations.NN = np.where(
    final_recommendations.graph_type == 0, final_recommendations.NN * 2, final_recommendations.NN
)
# final_recommendations.sort_values(by='Dataset', inplace=True)
# base_order = ['texture', 'moments', 'colorHisto', 'mnist', 'mnist121d', 'fashion', 'sift', 'cophir282', 'cophir64', 'base71'] 
# base_order = ['texture_67940', 'moments_67940', 'colorHisto_67940', 'mnist_69900', 'mnist121d_69900', 'fashion_69900', 'sift_999900', 'cophir282_999900', 'cophir64_999900', 'base71_999900',]
# if k_searching==10:
#     base_order.remove('base71')
# final_recommendations = final_recommendations[final_recommendations.Dataset.isin(base_order)]
# final_recommendations = final_recommendations.set_index('Dataset').loc[base_order, :].reset_index()
# final_recommendations = final_recommendations[final_recommendations.approach != 'tmmgs+']
final_recommendations.head()

,Dataset,NN,R,graph_type,approach,k_searching,Distance Computations,Query Time (ms),Recall,y_pred_DistComp,y_pred_QueryTime,y_pred_Recall,optmizing,required_recall
3735,base71_100000,70.0,40.0,1.0,GridSearch,1.0,13448.852316,0.591513,0.90,9.556739,0.647259,0.86,qt,0.9
10911,base71_500000,100.0,40.0,1.0,GridSearch,1.0,20696.700000,1.715990,0.90,9.882815,2.123658,0.88,qt,0.9
14973,base71_999900,140.0,10.0,0.0,GridSearch,1.0,21910.939801,3.728871,0.92,9.770167,2.193318,0.94,qt,0.9
21369,colorHisto_16000,70.0,1.0,1.0,GridSearch,1.0,441.558816,0.015332,0.94,6.020707,0.025917,0.93,qt,0.9
28209,colorHisto_32000,70.0,1.0,1.0,GridSearch,1.0,501.287890,0.023723,0.93,6.029161,0.028571,0.93,qt,0.9


In [28]:
final_recommendations = final_recommendations[final_recommendations.Dataset.isin(REAL_BASES)]
final_recommendations.to_csv(f'src/notebooks/overview/recommendations/final_recs_k={k_searching}.csv', index=False)

In [29]:
# Melt: columns to rows; Pivot: rows to columns

final_recommendations['Acertou'] = np.where(
    final_recommendations.Recall >= final_recommendations.required_recall, True, False
)

x=final_recommendations[~final_recommendations.isin(['Optimal'])].groupby('approach').Acertou.value_counts()
soma = final_recommendations[~final_recommendations.isin(['Optimal'])].approach.value_counts()
x.name = 'k'
x = x.reset_index(level=1)
a = (x.k/soma).to_frame()
a['Acertou'] = x.Acertou
a.reset_index(inplace=True)
a = a.rename(columns={0: 'rate', 'index': 'approach'})
a = a.append(pd.Series(['GridSearch', False, 0], index=['approach', 'Acertou', 'rate']).T, ignore_index=True)
a.Acertou = np.where(a.Acertou == True, 'Yes', 'No')
a = a.rename(columns={'Acertou': 'Satisfied'})
a.pivot(index='Satisfied', columns='approach', values='rate').loc[:, ['GridSearch', 'Light', 'Heavy', 'gmm', 'tmmgs', 'tmms', 'gmm+', 'tmmgs+', 'tmms+']].T['Yes'].sort_values(ascending=False)

approach
GridSearch    1.000000
gmm+          0.888889
tmmgs+        0.881481
Light         0.870370
tmmgs         0.859259
gmm           0.837037
tmms+         0.829630
Heavy         0.711111
tmms          0.644444
Name: Yes, dtype: float64

In [30]:
# fig, axes = plt.subplots(1,2, dpi=100, figsize=(18,4))
# axes = iter(axes.reshape(-1))
# for b in ['base71_100000', 'colorHisto_16000']:
#     ax = next(axes)
#     tmp = recommends[(recommends.base.str.startswith(b)) & (recommends.approach == 'gmm')].copy()
#     tmp['RecInt'] = pd.cut(recommends.y_true_Recall, bins=10)
#     tmp = pd.melt(tmp, id_vars='RecInt', value_vars=['y_true_Recall', 'y_pred_Recall'])
#     sns.boxplot(x='RecInt', y='value', hue='variable', data=tmp, ax=ax)
#     ax.set_title(b)